Reads and parses fNIRS data with traditional ML techniques

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
BASELINE_START = "baselinestart"
BASELINE_END = "baselineend"
EASY_START = "easystart"
EASY_END = "easyend"
HARD_START = "hardstart"
HARD_END = "hardend"

In [3]:
fnirs_path = os.path.join(os.getcwd(), "data/S902/2015-02-26_11-24-48-120", "fNIRSdata.txt")
marker_path = os.path.join(os.getcwd(), "data/S902/2015-02-26_11-24-48-120", "markers.txt")

First read the files

In [4]:
"""Gets the row blocks for easy and hard tasks
"""
def read_data(fnirs_path, marker_path):
    fnirs_df =  pd.read_csv(fnirs_path, sep='\t', skiprows=range(4), index_col=False)
    marker_df = pd.read_csv(marker_path, sep='\t', skiprows=range(4), index_col=False)
    
    merged_df = pd.merge(fnirs_df, marker_df, on="Matlab_now", how="left")
    
    
    return merged_df

In [5]:
def get_row_blocks(merged_df):
    easy_start_rows = merged_df.index[merged_df.Stimulus_Label == EASY_START].tolist()
    easy_end_rows = merged_df.index[merged_df.Stimulus_Label == EASY_END].tolist()
    hard_start_rows = merged_df.index[merged_df.Stimulus_Label == HARD_START].tolist()
    hard_end_rows = merged_df.index[merged_df.Stimulus_Label == HARD_END].tolist()
    
    easy_rows = list(zip(easy_start_rows, easy_end_rows))
    hard_rows = list(zip(hard_start_rows, hard_end_rows))
    
    return (easy_rows, hard_rows)

In [6]:
"""Return subset of df determined by the indices of the row blocks
"""
def get_subsets(merged_df, row_blocks):
    tables = []
    column_names = ["Matlab_now", "A-DC1", "A-DC2", "A-DC3", "A-DC4", "A-DC5",
                    "A-DC6", "A-DC7", "A-DC8", "B-DC1", "B-DC2", "B-DC3", 
                    "B-DC4", "B-DC5", "B-DC6", "B-DC7", "B-DC8"]
    column_indices = [merged_df.columns.get_loc(c) for c in column_names]
    for row_block in row_blocks:
        df = merged_df.iloc[row_block[0]:row_block[1], column_indices]
        start_time = df.iloc[0]["Matlab_now"]
        df["Matlab_now"] = df["Matlab_now"] - start_time

        tables.append(df)
    return tables

In [7]:
"""Perform linear fit and calculate linear fit coefficients and mean
    :param table: pandas df, subset to examine
    :return: Dictionary of
                key: Column name
                value: 3-tuple (a, b, mean), where y = ax + b
"""
def extract_feature(table):
    x = table["Matlab_now"].values
    cols = table.columns[1:]
    my_dict = {}
    for col in cols:
        y = table[col].values
        z = np.poly1d(np.polyfit(x, y, 1))
        my_tuple = (z[1], z[0], table[col].mean())

        my_dict[col] = my_tuple
    return my_dict

In [8]:
"""runs polyfit on the timeseries data
    :param tables: table of blocks of easy / hard tasks
    :param difficulty: 0 - easy, 1 - hard; labeling process
    
    :return: numpy array of
    feature_row: 
        AC-1 gradient, AC-1 intercept, AC-1 mean, AC-2 gradient ... DC-8 gradient DC-8 intercept DC-8 mean difficulty
    
    
    Dictionary of key: channel
                           value: (gradient, mean, difficulty)
"""
def extract_features(tables, difficulty):
    arr = np.empty((0, 49))
    for table in tables:
        x = table["Matlab_now"].values
        cols = table.columns[1:]
        feature_row = []
        for col in cols:
            y = table[col].values
            gradient, intercept = np.poly1d(np.polyfit(x, y, 1))
            avg = table[col].mean()
            
            feature_row = feature_row + [gradient, intercept, avg]
        feature_row.append(difficulty)
        feature_row = np.array(feature_row)
        
        arr = np.vstack([arr, feature_row])
    return arr
        

In [9]:
"""Extract features from given dataset
    :param data_path: Directory containing the files
    
    :return: gets all the easy and hard features from a given dataset
"""
def get_features_for_dataset(data_path):
    fnirs_path = os.path.join(os.getcwd(), data_path, "fNIRSdata.txt")
    marker_path = os.path.join(os.getcwd(), data_path, "markers.txt")
    merged_df = read_data(fnirs_path, marker_path)
    easy_rows, hard_rows = get_row_blocks(merged_df)
    
    easy_tables = get_subsets(merged_df, easy_rows)
    hard_tables = get_subsets(merged_df, hard_rows)
    easy_feature_rows = extract_features(easy_tables, 0)
    hard_feature_rows = extract_features(hard_tables, 1)
    
    features = np.vstack([easy_feature_rows, hard_feature_rows])

    return features

In [10]:
get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S902/2015-02-26_11-24-48-120")

array([[-1.98423547e+00,  1.88085852e+03,  1.85117232e+03, ...,
         1.25560231e+01,  1.23240395e+01,  0.00000000e+00],
       [-3.85560577e-01,  1.84479211e+03,  1.83903966e+03, ...,
         1.26342789e+01,  1.24279037e+01,  0.00000000e+00],
       [-1.47441570e+00,  1.86788622e+03,  1.84582203e+03, ...,
         1.23430331e+01,  1.23145198e+01,  0.00000000e+00],
       ...,
       [ 2.10889695e+00,  1.65531971e+03,  1.68682203e+03, ...,
         1.12960403e+01,  1.15924294e+01,  1.00000000e+00],
       [-8.11446930e-01,  1.70379200e+03,  1.69170822e+03, ...,
         1.15253817e+01,  1.16490652e+01,  1.00000000e+00],
       [-4.35653114e-01,  1.67908834e+03,  1.67258475e+03, ...,
         1.18857171e+01,  1.16263842e+01,  1.00000000e+00]])

In [11]:
data_path = "/Users/sjjin/workspace/hci_lab/data/S902/2015-02-26_11-24-48-120"
fnirs_path = os.path.join(os.getcwd(), data_path, "fNIRSdata.txt")
marker_path = os.path.join(os.getcwd(), data_path, "markers.txt")
merged_df = read_data(fnirs_path, marker_path)
easy_rows, hard_rows = get_row_blocks(merged_df)
easy_tables = get_subsets(merged_df, easy_rows)

In [12]:
len(easy_tables)

11

In [13]:
easy_tables[0]

,Matlab_now,A-DC1,A-DC2,A-DC3,A-DC4,A-DC5,A-DC6,A-DC7,A-DC8,B-DC1,B-DC2,B-DC3,B-DC4,B-DC5,B-DC6,B-DC7,B-DC8
1111,0.000,1885.0,478.9,131.2,40.20,1800.0,441.0,110.2,28.98,283.4,78.32,23.93,6.913,378.8,114.7,36.36,12.69
1112,0.062,1896.0,472.0,131.3,40.10,1805.0,440.4,110.3,29.07,283.1,78.39,23.95,6.923,379.3,114.9,36.57,12.72
1113,0.126,1882.0,478.3,131.8,40.22,1804.0,441.7,110.8,28.86,283.2,78.54,24.04,6.928,379.2,114.9,36.41,12.75
1114,0.264,1885.0,474.8,131.1,40.17,1802.0,442.2,110.1,29.03,283.4,78.16,23.86,6.901,376.4,114.2,36.34,12.63
1115,0.329,1886.0,476.7,130.4,39.66,1804.0,440.2,110.7,28.92,282.5,78.16,23.85,6.905,376.6,114.0,36.10,12.65
1116,0.467,1883.0,473.3,130.6,39.77,1809.0,440.9,109.9,29.02,281.5,78.29,23.98,6.884,377.2,114.0,36.19,12.61
1117,0.533,1883.0,479.3,130.4,40.07,1806.0,441.3,111.0,29.10,283.0,78.31,23.89,6.879,376.5,114.4,36.26,12.61
1118,0.608,1877.0,475.9,130.0,40.21,1797.0,443.5,110.7,28.91,282.0,78.47,23.93,6.918,376.9,114.0,36.28,12.63
1119,0.670,1880.0,474.1,130.7,40.00,1805.0,442.0,111.2,29.12,282.7,78.57,23.93,6.910,377.8,114.3,36.52,12.69
1120,0.732,1901.0,475.7,130.8,40.19,1803.0,443.0,111.1,29.14,285.5,78.50,24.07,6.917,378.7,114.7,36.44,12.67


In [14]:
"""Be able to merge a number of different datasets
    :param feature_dict_list: List of features to merge together
    
    :return merged dictionary
"""
def merge_features(feature_dict_list):
    main_dict = {}
    for feature_dict in feature_dict_list:
        for key in feature_dict.keys():
            if key in main_dict:
                main_dict[key] = main_dict[key] + feature_dict[key]
            else:
                main_dict[key] = feature_dict[key]
    return main_dict

In [15]:
features_902 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S902/2015-02-26_11-24-48-120")
features_903 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S903/2015-02-27_13-20-42-120")
features_904 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S904/2015-02-27_15-30-27-120")
features_905 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S905/2015-03-02_13-14-35-120")
features_906 = get_features_for_dataset("/Users/sjjin/workspace/hci_lab/data/S906/2015-03-05_11-17-38-120")

train_set = np.vstack([features_902, features_903, features_904, features_905])
test_set = features_906

In [16]:
train_set_x = train_set[:, :-1]
train_set_y = train_set[:, -1]
test_set_x = test_set[:, :-1]
test_set_y = test_set[:, -1]

In [28]:
pd = pd.

array([[-1.98423547e+00,  1.88085852e+03,  1.85117232e+03, ...,
        -1.55058519e-02,  1.25560231e+01,  1.23240395e+01],
       [-3.85560577e-01,  1.84479211e+03,  1.83903966e+03, ...,
        -1.38323870e-02,  1.26342789e+01,  1.24279037e+01],
       [-1.47441570e+00,  1.86788622e+03,  1.84582203e+03, ...,
        -1.90537288e-03,  1.23430331e+01,  1.23145198e+01],
       ...,
       [-2.39934402e+00,  1.14923396e+03,  1.11336158e+03, ...,
        -3.89149994e-02,  6.85373848e+00,  6.27192373e+00],
       [-3.33724264e+00,  1.11290992e+03,  1.06299718e+03, ...,
        -4.19361535e-02,  6.48779653e+00,  5.86058757e+00],
       [-5.67515709e-01,  1.08201662e+03,  1.07351412e+03, ...,
        -2.11510457e-02,  6.40478226e+00,  6.08789831e+00]])

In [17]:
len(train_set_y)

88

In [18]:
len(test_set_x)

21

In [19]:
clf = SVC(gamma='auto')
clf.fit(train_set_x, train_set_y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
test_set_x.shape

(21, 48)

In [21]:
result = clf.predict(test_set_x)
result

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [22]:
accuracy_score(test_set_y, result)

0.5238095238095238

In [23]:
"""
"""
def plot_feature(table, feature_dict, column_name):
    x = table["Matlab_now"]
    y1 = table[column_name]
    feature = feature_dict[column_name]
    y2 = feature[0] * x + feature[1]

    plt.plot(x, y1)
    plt.plot(x, y2)
    plt.xlabel("time(s)")
    plt.ylabel("Light intensity")
    plt.title(column_name)
    plt.show()

In [24]:
"""
"""
def plot_feature(table, feature_dict, column_name):
    x = table["Matlab_now"]
    y1 = table[column_name]
    feature = feature_dict[column_name]
    y2 = feature[0] * x + feature[1]

    fig = plt.figure()
    ax = ax = fig.add_subplot(111)
    ax.plot(x, y1)
    ax.plot(x, y2)
    ax.set_xlabel("time(s)")
    ax.set_ylabel("Light intensity")
    ax.set_title(column_name)
    print(get_axis_limits(ax))
    #ax.annotate("text", get_axis_limits(ax))
    plt.show()

In [25]:
def get_axis_limits(ax, scale=0.9):
    return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale

In [26]:
def plot_all_features(table, feature_dict):
    x = table["Matlab_now"]
    fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(45, 45))

    count = 0
    for row in ax:
        for col in row:
            if (count == len(table.columns[1:].values)):
                break
            column_name = table.columns[1:].values[count]
    
            feature = feature_dict[column_name]
            
            y1 = table[column_name]
            y2 = feature[0] * x + feature[1]
            #y3 = [feature[2] for _ in range(len(y1))]
            

            col.plot(x, y1)
            col.plot(x, y2)
            col.set_xlabel("time(s)")
            col.set_ylabel("Light intensity")
            col.title.set_text(column_name)
            #my_text1 = "y = %f*x + %f" % (feature[0], feature[1])
            #my_text2 = "mean = %f" % (feature[2])
            my_text1 = "y = {:9.4f}*x + {:9.4f}".format(feature[0], feature[1])
            my_text2 = "mean = {:9.4f}".format(feature[2])
            col.text(0.3, 0.9, my_text1 , transform=col.transAxes, size=20, weight='bold')
            col.text(0.3, 0.8, my_text2 , transform=col.transAxes, size=20, weight='bold')

            count += 1
    #return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale
    #ax1.annotate('A', xy=get_axis_limits(ax1))
    plt.savefig("fig1.png")
    plt.show()
